In [1]:
import os, sys
import pandas as pd
from glob import glob
import requests
from datetime import date
today = date.today()
print ( "Today's date:", today)

Today's date: 2023-12-05


In [9]:
# data loading 
path = '/Users/barradd/Documents/GitHub/machine_learning_chem_RGS/data/inital-data-19-nov-23.xlsx'
my_df = pd.read_excel(path)

In [3]:
def query_pubchem_by_name(name):
  """Queries the PubChem database using a molecular formula query.

  Args:
    molecular_formula: A string representing the molecular formula.

  Returns:
    A list of PubChem compound IDs of the compounds that match the molecular formula query.
  """
### this is the correct way to use the fastformula 
  url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/{name}/cids/JSON"



  # Make a GET request to the URL.
  response = requests.get(url)

  # Parse the JSON response.
  json_response = response.json()

    # Check if the response is successful.
  if response.status_code == 200:
    # Parse the JSON response.
    json_response = response.json()

    # Extract the PubChem compound IDs of the compounds that match the query.
    compound_ids = []
    for compound in json_response["IdentifierList"]["CID"]:
      compound_ids.append(compound)

    # Return the list of compound IDs.
    return compound_ids
  else:
    # Return an empty list if there was an error.
    return []


In [4]:
import pubchempy as pcp 
from rdkit import Chem


In [10]:
compound_cids = {} 
for compound_raw_name in my_df['Name'] : 
    # print (compound_raw_name)
    compound_raw_name = compound_raw_name.strip()
    compound_ids  = query_pubchem_by_name(compound_raw_name)
    compound_cids[compound_raw_name] = compound_ids


In [69]:
name_set_1, name_set_2 = compound_cids.keys(), my_df['Name'].to_list()
for num, m in enumerate ( name_set_2) :
    for n in list (name_set_1):
        if m == n :
            print ( num ,m, n , compound_cids[m] )

0 Benzenesulfonyl cyanide Benzenesulfonyl cyanide [10313279]
1 2-phenylethene-1,1,2-tricarbonitrile 2-phenylethene-1,1,2-tricarbonitrile [285442]
2 benzenesulfonyl fluoride benzenesulfonyl fluoride [67779]
3 benzenesulfonyl chloride benzenesulfonyl chloride [7369]
4 Trinitromethylbenzene Trinitromethylbenzene [12503820]
5 Phenyl trifluoromethyl sulfone Phenyl trifluoromethyl sulfone [555605]
6 (Pentafluoroethyl)phenyl sulfone (Pentafluoroethyl)phenyl sulfone [12599780]
7 (NE)-N-benzylidene-1,1,1-trifluoromethanesulfonamide (NE)-N-benzylidene-1,1,1-trifluoromethanesulfonamide [13678597]
8 ((Difluoromethyl)sulfonyl)benzene ((Difluoromethyl)sulfonyl)benzene [11816356]
9 Phenylmethanetricarbonitrile Phenylmethanetricarbonitrile [20304470]
10 Benzylidenemalononitrile Benzylidenemalononitrile [17608]
11 phenylsulfur trifluoride phenylsulfur trifluoride [14694018]
12 Phenylphosphonicdifluoride Phenylphosphonicdifluoride [595499]
13 Phenyltetrafluorophosphorane Phenyltetrafluorophosphorane [69

In [22]:
len(name_set_1),len(name_set_2)

(343, 350)

In [27]:
def checkKey(dic, key):
     
    if key in dic:
        pass
        # print("Present, ", end =" ")
        # print("value =", dic[key])
    else:
        print(f"Not present {key}")

In [39]:
counter0 , counter1 = 0 , 0 
for num,name in enumerate( name_set_2) :
    checkKey(compound_cids, name)
    if len(compound_cids[name]) > 0 :
        counter1+= len(compound_cids[name])

    else :
        counter0+=1
        print (num,name ,compound_cids[name])

print (counter0 , counter1)

47 1,1,1,3,3,3-hexafluoro-2-(trifluoromethyl)propan-2-yl]sulfanylbenzene []
83 1-fluoro-4-(1,1,2,2-tetrafluoro-2-phenylethyl)benzene []
167 (phenyl)sulfanylidene-lambda~5~-phosphane []
173 (4-Methylphenyl)(oxo)diphenyl-lambda~5~phosphane []
175 Chloro(phenyl)(dimethylamino)phosphine []
177 chloroethenyl]selanylbenzene []
254 diethyl(phenyl)arsane []
7 345


In [42]:
for num,name in enumerate( name_set_2) :
    checkKey(compound_cids, name)
    if len(compound_cids[name]) > 1 :
        print ( name ,len(compound_cids[name]) ) 


(Z)-Benzaldehyde oxime 3


In [43]:
my_properties_list = [ 
    'cid',
    'molecular_formula',
    'molecular_weight',
    'exact_mass',
    'monoisotopic_mass' ,
    'charge' ,
    'heavy_atom_count',
    'h_bond_donor_count',
    'h_bond_acceptor_count',
    'xlogp',
    'tpsa',
    'canonical_smiles',
    'complexity',
    'covalent_unit_count',
    'bonds',
    'elements',
    'iupac_name'
                      ] 

In [44]:
import time

def get_compound_info(my_cids):
    while True:
        try:
            c = pcp.Compound.from_cid(my_cids)
            return c
        except pcp.PubChemHTTPError as e:
            if e.msg == "PUGREST.ServerBusy":
        # except urllib.error.HTTPError as e:
        #     if e.code == 503:
                print("Server busy, waiting for 10 seconds...")
                time.sleep(10)  # Wait for 10 seconds before retrying
            else:
                raise e


In [45]:
def get_the_rdkit_features(df_to_edit):
    mol = Chem.MolFromSmiles(df_to_edit['canonical_smiles'][0])
    try:
        df_to_edit['Num_of_Rings'] = mol.GetRingInfo().NumRings()
    except AttributeError:
        # Handle the case where mol has no attribute 'GetRingInfo'
        print("Warning: Mol has no attribute 'GetRingInfo' for molecule:", df_to_edit['canonical_smiles'][0])
        print ("Setting Num_of_Rings to 0")
        df_to_edit['Num_of_Rings'] = 0

    df_to_edit['heavy_atoms'] = mol.GetNumAtoms()
    return df_to_edit, mol 

In [46]:
def get_the_type_of_bond(mol,my_cids):
    counter_ar, counter_single, counter_double, counter_triple = 0, 0, 0, 0

    try:
        for num, m in enumerate(mol.GetAtoms()):
            # Retrieve bond information for the current atom
            bond_type = str(mol.GetBonds()[num].GetBondType())

            # Update bond type counters
            if bond_type == "AROMATIC":
                counter_ar += 1
            elif bond_type == "SINGLE":
                counter_single += 1
            elif bond_type == "DOUBLE":
                counter_double += 1
            else:
                counter_triple += 1

    except IndexError:
        # Handle out-of-range error
        print(f"Error: Index out of range when accessing bond information, CIDS: {my_cids}")
        # exit()
        # return None

    # Create and return a DataFrame containing the bond type counts
    df_temp = pd.DataFrame([counter_ar, counter_single, counter_double, counter_triple],
                           index=['aromatic_bond', 'single_bond', 'double_bond', 'triple_bond'])
    return df_temp.T

In [47]:
def feature_eng(dataframe, my_cid):
    element_type = pd.get_dummies( pd.Series ( dataframe['elements'].iloc[0]) ,dtype=float ).sum()
    dataframe['all_atoms_count']= dataframe['elements'].apply( lambda x:len(x))
    dataframe['all_atoms_count_unique'] = dataframe['elements'].apply( lambda x: len ( list(dict.fromkeys(x)) ) )

    df_to_edit_2 , mol  = get_the_rdkit_features(df_to_edit=dataframe)
    df_to_edit_3 = get_the_type_of_bond(mol,my_cid)

    final_dataframe = pd.concat( [df_to_edit_2, df_to_edit_3,element_type.to_frame().T],axis=1)
    final_dataframe.drop(['bonds','elements'],axis=1 ,inplace=True)
    return final_dataframe

In [48]:
cids_missing = {
                "diethyl(phenyl)arsane":592868,
                "chloroethenyl]selanylbenzene":13334225,
                "Chloro(phenyl)(dimethylamino)phosphine":12929704, 
                "(4-Methylphenyl)(oxo)diphenyl-lambda~5~phosphane":12550549,
                "(phenyl)sulfanylidene-lambda~5~-phosphane":13799698,
                "1-fluoro-4-(1,1,2,2-tetrafluoro-2-phenylethyl)benzene":9993363,
                "1,1,1,3,3,3-hexafluoro-2-(trifluoromethyl)propan-2-yl]sulfanylbenzene":4171287}

In [79]:
all_data_frames, no_cids = [],[]
# for key,my_cids in compound_cids.items():
for key  in my_df['Name'].to_list() : 
    my_cids = compound_cids[key]
    if len (my_cids) > 0 :
        # print (key,my_cids[0] )
        c = get_compound_info(my_cids[0])
        # get the information on a dictionary 
        temp_dict = c.to_dict( properties= my_properties_list) 
        # transform the dict into pandas dataframe 
        df_to_edit = pd.DataFrame.from_dict(data=temp_dict,orient='index').T
        df_temp = feature_eng(dataframe=df_to_edit, my_cid=my_cids[0])
        df_temp["Name"]=key
        all_data_frames.append(df_temp)
    else :
        # print (key,cids_missing[key] )

        no_cids.append(key)
        c = get_compound_info(cids_missing[key])
        temp_dict = c.to_dict( properties= my_properties_list) 
      
        # transform the dict into pandas dataframe 
        df_to_edit = pd.DataFrame.from_dict(data=temp_dict,orient='index').T
        df_temp = feature_eng(dataframe=df_to_edit, my_cid=cids_missing[key])
        df_temp["Name"]=key

        all_data_frames.append(df_temp)
        # no_cids.append(df_temp)






In [80]:
print (f"elements with no cids {len(no_cids)} , number with info {len(all_data_frames)}")

elements with no cids 7 , number with info 350


In [81]:
no_cids


['1,1,1,3,3,3-hexafluoro-2-(trifluoromethyl)propan-2-yl]sulfanylbenzene',
 '1-fluoro-4-(1,1,2,2-tetrafluoro-2-phenylethyl)benzene',
 '(phenyl)sulfanylidene-lambda~5~-phosphane',
 '(4-Methylphenyl)(oxo)diphenyl-lambda~5~phosphane',
 'Chloro(phenyl)(dimethylamino)phosphine',
 'chloroethenyl]selanylbenzene',
 'diethyl(phenyl)arsane']

In [82]:
df_pubchem = pd.concat(all_data_frames, axis=0)

In [83]:

counter = 0
for col in df_pubchem.columns.to_list():
    if counter > 10 :
        print ()
        counter =0 
    else :
        print (col , end=" ")
        counter+=1

cid molecular_formula molecular_weight exact_mass monoisotopic_mass charge heavy_atom_count h_bond_donor_count h_bond_acceptor_count xlogp tpsa 
complexity covalent_unit_count iupac_name all_atoms_count all_atoms_count_unique Num_of_Rings heavy_atoms aromatic_bond single_bond double_bond triple_bond 
H N O S Name F Cl P Ge Se Si 
B As 

In [84]:
cid_found = df_pubchem["cid"].to_list()
cid_original = [ item[0] for key, item in compound_cids.items() if len(item) >0  ]

In [85]:
350 - len(cid_original)

14

In [86]:
df_pubchem.describe()

,all_atoms_count,all_atoms_count_unique,Num_of_Rings,heavy_atoms,aromatic_bond,single_bond,double_bond,triple_bond,C,H,...,S,F,Cl,P,Ge,Se,Si,Br,B,As
count,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,...,78.000000,69.000000,28.000000,37.0,6.0,5.0,21.000000,8.000000,2.0,4.0
mean,21.568571,3.511429,1.297143,11.945714,7.257143,4.011429,0.600000,0.077143,9.437143,9.622857,...,1.038462,3.521739,1.678571,1.0,1.0,1.0,1.095238,1.875000,1.0,1.0
std,6.512024,0.842044,0.589053,3.301458,2.722535,2.371001,0.772003,0.350673,3.138684,4.367835,...,0.193552,2.055185,0.983327,0.0,0.0,0.0,0.436436,0.991031,0.0,0.0
min,12.000000,2.000000,1.000000,6.000000,5.000000,0.000000,0.000000,0.000000,6.000000,5.000000,...,1.000000,1.000000,1.000000,1.0,1.0,1.0,1.000000,1.000000,1.0,1.0
25%,17.000000,3.000000,1.000000,10.000000,6.000000,2.000000,0.000000,0.000000,7.000000,6.000000,...,1.000000,2.000000,1.000000,1.0,1.0,1.0,1.000000,1.000000,1.0,1.0
50%,20.000000,4.000000,1.000000,11.000000,6.000000,4.000000,0.000000,0.000000,8.000000,9.000000,...,1.000000,3.000000,1.000000,1.0,1.0,1.0,1.000000,1.500000,1.0,1.0
75%,25.000000,4.000000,1.000000,14.000000,6.000000,5.000000,1.000000,0.000000,11.000000,12.000000,...,1.000000,5.000000,2.000000,1.0,1.0,1.0,1.000000,3.000000,1.0,1.0
max,45.000000,6.000000,5.000000,25.000000,21.000000,14.000000,4.000000,3.000000,25.000000,26.000000,...,2.000000,9.000000,5.000000,1.0,1.0,1.0,3.000000,3.000000,1.0,1.0


In [87]:
df_pubchem.info()

<class 'pandas.core.frame.DataFrame'>
Index: 350 entries, 0 to 0
Data columns (total 38 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   cid                     350 non-null    object 
 1   molecular_formula       350 non-null    object 
 2   molecular_weight        350 non-null    object 
 3   exact_mass              350 non-null    object 
 4   monoisotopic_mass       350 non-null    object 
 5   charge                  350 non-null    object 
 6   heavy_atom_count        350 non-null    object 
 7   h_bond_donor_count      350 non-null    object 
 8   h_bond_acceptor_count   350 non-null    object 
 9   xlogp                   312 non-null    object 
 10  tpsa                    350 non-null    object 
 11  canonical_smiles        350 non-null    object 
 12  complexity              350 non-null    object 
 13  covalent_unit_count     350 non-null    object 
 14  iupac_name              348 non-null    object 
 

In [88]:
my_df.columns = ['substituent', 'molecular_formula', 'Name', 'ΔVC-m', 'ΔVC -p', 'σm','σp', 'Fb', 'Rc']

In [90]:
df_merged = pd.merge(left=my_df,right=df_pubchem,on="Name")

In [91]:
df_merged.head()

,substituent,molecular_formula_x,Name,ΔVC-m,ΔVC -p,σm,σp,Fb,Rc,cid,...,S,F,Cl,P,Ge,Se,Si,Br,B,As
0,SO2CN,C7H5NO2S,Benzenesulfonyl cyanide,27.007403,28.300000,1.10,1.26,0.97,0.29,10313279,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,C(CN)=C(CN)2,C11H5N3,"2-phenylethene-1,1,2-tricarbonitrile",24.873869,27.000000,0.77,0.98,0.65,0.33,285442,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SO2F,C6H5FO2S,benzenesulfonyl fluoride,24.270832,25.381524,0.80,0.91,0.72,0.19,67779,...,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,SO2Cl,C6H5ClO2S,benzenesulfonyl chloride,24.246359,25.200000,1.20,1.11,1.16,-0.05,7369,...,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,C(NO2)3,C7H5N3O6,Trinitromethylbenzene,23.556098,25.100000,0.72,0.82,0.65,0.17,12503820,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [92]:
df_merged.shape

(364, 46)

In [97]:
df_merged.to_excel(f"../data/pubchem_data_{today}_{len(all_data_frames)}_samples.xlsx", index=False)

In [93]:
df_merged.to_csv(f"../data/pubchem_data_{today}_{len(all_data_frames)}_samples.csv", index=False)

In [96]:
df_merged.tail()


,substituent,molecular_formula_x,Name,ΔVC-m,ΔVC -p,σm,σp,Fb,Rc,cid,...,S,F,Cl,P,Ge,Se,Si,Br,B,As
359,NHEt,C8H11N,Ethylaniline,-5.497615,-11.4,-0.24,-0.61,-0.04,-0.57,7670,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
360,NH(CH2)3CH3,C10H15N,N-Butylaniline,-5.685868,-11.6,-0.34,-0.51,-0.21,-0.30,14310,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
361,N(Et)2,C10H15N,Diethylaniline,-6.313378,-11.9,-0.23,-0.72,0.01,-0.73,7061,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
362,N(Me)2,C8H11N,"N,N-dimethylaniline",-6.438880,-12.3,-0.16,-0.83,0.15,-0.98,949,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
363,N(C3H7)2,C12H19N,"N,N-Dipropylaniline",-6.940888,-12.9,-0.26,-0.93,0.06,-0.99,75191,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [95]:
df_merged.describe()

,ΔVC-m,ΔVC -p,σm,σp,Fb,Rc,all_atoms_count,all_atoms_count_unique,Num_of_Rings,heavy_atoms,...,S,F,Cl,P,Ge,Se,Si,Br,B,As
count,364.000000,364.000000,364.000000,364.000000,364.000000,364.00000,364.000000,364.000000,364.000000,364.000000,...,82.000000,69.000000,28.000000,39.0,6.0,5.0,23.000000,8.000000,2.0,4.0
mean,6.623192,6.225898,0.258654,0.229588,0.277692,-0.05500,21.662088,3.519231,1.307692,11.991758,...,1.036585,3.521739,1.678571,1.0,1.0,1.0,1.086957,1.875000,1.0,1.0
std,7.010619,8.068139,0.250236,0.349373,0.211739,0.22673,6.430694,0.837919,0.588078,3.278184,...,0.188897,2.055185,0.983327,0.0,0.0,0.0,0.417029,0.991031,0.0,0.0
min,-6.940888,-12.900000,-0.340000,-0.930000,-0.350000,-0.99000,12.000000,2.000000,1.000000,6.000000,...,1.000000,1.000000,1.000000,1.0,1.0,1.0,1.000000,1.000000,1.0,1.0
25%,0.714734,0.200000,0.060000,0.000000,0.120000,-0.15000,17.000000,3.000000,1.000000,10.000000,...,1.000000,2.000000,1.000000,1.0,1.0,1.0,1.000000,1.000000,1.0,1.0
50%,6.017193,6.100000,0.250000,0.200000,0.280000,-0.03000,21.000000,4.000000,1.000000,11.000000,...,1.000000,3.000000,1.000000,1.0,1.0,1.0,1.000000,1.500000,1.0,1.0
75%,11.892883,11.325000,0.412500,0.480000,0.400000,0.10000,25.000000,4.000000,2.000000,14.000000,...,1.000000,5.000000,2.000000,1.0,1.0,1.0,1.000000,3.000000,1.0,1.0
max,27.007403,28.300000,1.200000,1.260000,1.160000,0.45000,45.000000,6.000000,5.000000,25.000000,...,2.000000,9.000000,5.000000,1.0,1.0,1.0,3.000000,3.000000,1.0,1.0


In [98]:
list_of_names_after_merge = df_merged["Name"].to_list()

In [102]:
repeated_compuds = []
for name in df_merged["Name"].to_list():
    count = list_of_names_after_merge.count(name)
    if count >1 :
        print ( name, count )
        if name not in repeated_compuds:
            repeated_compuds.append(name)

N-Phenylbenzenesulfonamide 4
N-Phenylbenzenesulfonamide 4
N-Phenylbenzenesulfonamide 4
N-Phenylbenzenesulfonamide 4
Phenyl benzoate 4
Phenyl benzoate 4
Phenyl benzoate 4
Phenyl benzoate 4
Benzanilide 4
Benzanilide 4
Benzanilide 4
Benzanilide 4
N-Benzylideneaniline 4
N-Benzylideneaniline 4
N-Benzylideneaniline 4
N-Benzylideneaniline 4
Dimethyl Phenylphosphonate 4
Dimethyl Phenylphosphonate 4
Dimethyl Phenylphosphonate 4
Dimethyl Phenylphosphonate 4
Phenyl vinyl sulfide 4
Phenyl vinyl sulfide 4
Phenyl vinyl sulfide 4
Phenyl vinyl sulfide 4
Methoxydimethylphenylsilane 4
Methoxydimethylphenylsilane 4
Methoxydimethylphenylsilane 4
Methoxydimethylphenylsilane 4


In [103]:
df_merged [ df_merged["Name"].isin(repeated_compuds) ]

,substituent,molecular_formula_x,Name,ΔVC-m,ΔVC -p,σm,σp,Fb,Rc,cid,...,S,F,Cl,P,Ge,Se,Si,Br,B,As
46,SO2NHC6H5,C12H11NO2S,N-Phenylbenzenesulfonamide,15.147464,15.80000,0.56,0.65,0.51,0.14,74296,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47,SO2NHC6H5,C12H11NO2S,N-Phenylbenzenesulfonamide,15.147464,15.80000,0.56,0.65,0.51,0.14,74296,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48,NHSO2C6H5,C12H11NO2S,N-Phenylbenzenesulfonamide,3.601280,4.80000,0.16,0.01,0.24,-0.23,74296,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49,NHSO2C6H5,C12H11NO2S,N-Phenylbenzenesulfonamide,3.601280,4.80000,0.16,0.01,0.24,-0.23,74296,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
109,COOC6H5,C13H10O2,Phenyl benzoate,7.805597,10.00000,0.37,0.44,0.34,0.10,7169,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
110,COOC6H5,C13H10O2,Phenyl benzoate,7.805597,10.00000,0.37,0.44,0.34,0.10,7169,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
111,OCOC6H5,C13H10O2,Phenyl benzoate,1.091240,2.30000,0.21,0.13,0.26,-0.13,7169,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
112,OCOC6H5,C13H10O2,Phenyl benzoate,1.091240,2.30000,0.21,0.13,0.26,-0.13,7169,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
121,CONHC6H5,C13H11NO,Benzanilide,8.621360,9.30000,0.23,0.41,0.17,0.24,7168,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
122,CONHC6H5,C13H11NO,Benzanilide,8.621360,9.30000,0.23,0.41,0.17,0.24,7168,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [104]:
repeated_compuds

['N-Phenylbenzenesulfonamide',
 'Phenyl benzoate',
 'Benzanilide',
 'N-Benzylideneaniline',
 'Dimethyl Phenylphosphonate',
 'Phenyl vinyl sulfide',
 'Methoxydimethylphenylsilane']